## X infinite poster powered by SLM
In this example we use Assisterr SLM model that can craft X posts from web3 structured data to automatically create X posts based on latest web3 news.

In [1]:
import requests
import tweepy
import json
import time


# take these from developer.twitter.com
API_KEY = ""
API_SECRET = ""
ACCESS_TOKEN = ""
ACCESS_SECRET = ""
BEARER_TOKEN = ""



# API Key can be obtained from https://build.assisterr.ai/ailab 
ASSISTERR_API_KEY = ''
ASSISTERR_SLM_X_POSTER = 'x_poster'

In [2]:
# For this example we'll use cryptopanic.com API to extract latest news from Web3
# To enable this example grab you free API key from https://cryptopanic.com/developers/api/keys/

CRYPTOPANIC_API_KEY = ''
def make_web3_call():

    url = f"https://cryptopanic.com/api/v1/posts/?auth_token={CRYPTOPANIC_API_KEY}&kind=news&regions=en"

    headers = {
      "Accept": "application/json"
    }
    
    response = requests.request("GET", url, headers=headers)
    
    metadata = [{'source': i['source']['title'], 'news': i['title']} for i in response.json()['results'][:10]] # Limit to 10 just to make sample smaller.
    
    data = {'description': 'Data represents some latest web3 news', 'data': metadata}
    
    return data

In [3]:
# Then we need to call Assisterr SLM to convert raw web3 data into X post

def make_slm_call(slm_slug: str, query: str):

    url = f'https://api.assisterr.ai/api/v1/slm/{slm_slug}/chat/'

    response = requests.post(url=url, headers={'X-Api-Key': ASSISTERR_API_KEY}, json={'query': query})
    print(response.text)

    return response.json()['message']

In [4]:
# Finally post Message that Assisterr SLM created on X

def post_tweet(message):
        
    newapi = tweepy.Client(
        bearer_token=BEARER_TOKEN,
        access_token=ACCESS_TOKEN,
        access_token_secret=ACCESS_SECRET,
        consumer_key=API_KEY,
        consumer_secret=API_SECRET,
    )
    
    post_result = newapi.create_tweet(text=message)
    print(f"Tweet posted: {post_result.data['text']}")

In [5]:
# Combine it all together:

data = json.dumps(make_web3_call())
slm = make_slm_call(ASSISTERR_SLM_X_POSTER, data)
post_tweet(slm)

In [6]:
# Bonus section! Infinite news poster on X empowered by Assisterr SLM

def infinite_poster(time_interval=5*60*60):
    while True:
        data = json.dumps(make_web3_call())
        slm = make_slm_call(ASSISTERR_SLM_X_POSTER, data)
        post_tweet(slm)
        time.sleep(time_interval)

In [7]:
infinite_poster()